## **Note!**

Make sure you install pygame in your terminal: \
pip install pygame

## **Prototype 8 - Final + Sound effect**

In [ ]:
import pygame
import sys
import random
import heapq
from datetime import datetime, timedelta
from collections import deque

# --- Initialize Pygame & Mixer ---
pygame.init()
pygame.mixer.init()

# --- Set up Display (required before convert_alpha) ---
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Mega Maze Pathfinder with Sound")
clock = pygame.time.Clock()

# --- Load Sound Effects ---
pickup_sound      = pygame.mixer.Sound('pickup.wav')
place_block_sound = pygame.mixer.Sound('place_block.wav')
game_over_sound   = pygame.mixer.Sound('game_over.wav')

# Background music (looped)
pygame.mixer.music.load('bg_music.mp3')
pygame.mixer.music.set_volume(0.4)
pygame.mixer.music.play(-1)

# --- Colors & Constants ---
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
ORANGE = (255, 165, 0)
LIGHT_BROWN = (205, 133, 63)

CELL_SIZE = 30
COLS = 50
ROWS = HEIGHT // CELL_SIZE
AGENT_SPEED = 250
MIN_DISTANCE = COLS // 4
ITEM_TYPES = {
    'speed_up':    {'color': ORANGE, 'effect': 1.5,  'duration': 2, 'image_key': 'accelerator'},
    'speed_down':  {'color': RED,    'effect': 0.67, 'duration': 2, 'image_key': 'shackle'},
    'reveal_path': {'color': (128,0,128), 'effect': True, 'duration': 2, 'image_key': 'god_eye'}
}
ITEM_SPACING = 3
NUM_ITEMS = 25
ITEM_SCALE = 0.7
CAMERA_LERP = 0.1
CAMERA_OFFSET = WIDTH // 4
MAX_CAMERA_X = (COLS * CELL_SIZE) - WIDTH

# --- Load & Scale Images ---
try:
    thief_img       = pygame.image.load('Thief.PNG').convert_alpha()
    wall_img        = pygame.image.load('Wall.PNG').convert_alpha()
    altar_img       = pygame.image.load('Altar.PNG').convert_alpha()
    blue_portal_img = pygame.image.load('BluePortal.PNG').convert_alpha()
    accelerator_img = pygame.image.load('Accelerator.PNG').convert_alpha()
    shackle_img     = pygame.image.load('Shackle.PNG').convert_alpha()
    god_eye_img     = pygame.image.load('GodEye.PNG').convert_alpha()
except pygame.error as e:
    print(f"Error loading images: {e}")
    pygame.quit()
    sys.exit()

# Scale images to fit cells
thief_img       = pygame.transform.scale(thief_img, (CELL_SIZE, CELL_SIZE))
wall_img        = pygame.transform.scale(wall_img, (CELL_SIZE, CELL_SIZE))
altar_img       = pygame.transform.scale(altar_img, (CELL_SIZE, CELL_SIZE))
blue_portal_img = pygame.transform.scale(blue_portal_img, (CELL_SIZE, CELL_SIZE))
item_w = int(CELL_SIZE * ITEM_SCALE)
item_h = int(CELL_SIZE * ITEM_SCALE)
accelerator_img = pygame.transform.scale(accelerator_img, (item_w, item_h))
shackle_img     = pygame.transform.scale(shackle_img, (item_w, item_h))
god_eye_img     = pygame.transform.scale(god_eye_img, (item_w, item_h))
ITEM_IMAGES = {
    'accelerator': accelerator_img,
    'shackle':     shackle_img,
    'god_eye':     god_eye_img
}

# --- Classes ---
class Item:
    def __init__(self, pos, type_key):
        self.grid_pos   = pos
        self.screen_pos = (pos[1]*CELL_SIZE + CELL_SIZE//2,
                           pos[0]*CELL_SIZE + CELL_SIZE//2)
        self.type       = type_key
        self.active     = True

    @staticmethod
    def check_item_collision(agent_pos, items_list):
        rect = pygame.Rect(agent_pos.x - CELL_SIZE//4,
                           agent_pos.y - CELL_SIZE//4,
                           CELL_SIZE//2, CELL_SIZE//2)
        for item in items_list:
            if not item.active:
                continue
            w = CELL_SIZE * ITEM_SCALE
            irect = pygame.Rect(item.screen_pos[0]-w//2,
                                 item.screen_pos[1]-w//2,
                                 w, w)
            if rect.colliderect(irect):
                item.active = False
                return item
        return None

class SmoothAgent:
    def __init__(self, start_pos):
        self.target_pos     = start_pos
        self.current_pos    = pygame.Vector2(
            start_pos[1]*CELL_SIZE + CELL_SIZE//2,
            start_pos[0]*CELL_SIZE + CELL_SIZE//2
        )
        self.base_speed     = AGENT_SPEED
        self.speed          = self.base_speed
        self.active_effects = deque(maxlen=3)

    def update(self, path, dt):
        if not path:
            return
        # Move towards current target cell
        target = pygame.Vector2(
            self.target_pos[1]*CELL_SIZE + CELL_SIZE/2,
            self.target_pos[0]*CELL_SIZE + CELL_SIZE/2
        )
        direction = target - self.current_pos
        if direction.length() > 0:
            move = self.speed * dt
            if direction.length() <= move:
                self.current_pos = target
            else:
                direction.scale_to_length(move)
                self.current_pos += direction
            if (self.current_pos - target).length() < 1:
                self.current_pos = target

# --- Maze & Pathfinding Functions ---
def heuristic(a, b): return abs(a[0]-b[0]) + abs(a[1]-b[1])

def get_neighbors(maze, pos):
    dirs = [(-1,0),(1,0),(0,-1),(0,1)]
    r,c = pos
    return [(r+dr,c+dc) for dr,dc in dirs
            if 0<=r+dr<ROWS and 0<=c+dc<COLS and maze[r+dr][c+dc]==0]

def reconstruct_path(came_from, current):
    path=[current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    return path[::-1]

def astar(maze, start, end):
    open_set=[]
    heapq.heappush(open_set,(0,start))
    came_from={}
    g={ (r,c):float('inf') for r in range(ROWS) for c in range(COLS) }
    g[start]=0
    f=g.copy()
    f[start]=heuristic(start,end)
    open_hash={start}
    while open_set:
        _, current = heapq.heappop(open_set)
        if current==end:
            return reconstruct_path(came_from,current)
        open_hash.discard(current)
        for nbr in get_neighbors(maze,current):
            tg = g[current]+1
            if tg<g[nbr]:
                came_from[nbr]=current
                g[nbr]=tg
                f[nbr]=tg+heuristic(nbr,end)
                if nbr not in open_hash:
                    heapq.heappush(open_set,(f[nbr],nbr))
                    open_hash.add(nbr)
    return None

def bfs_reachable(maze, start):
    visited={start}
    dq=deque([start])
    dirs=[(-1,0),(1,0),(0,-1),(0,1)]
    while dq:
        r,c=dq.popleft()
        for dr,dc in dirs:
            nr,nc=r+dr,c+dc
            if 0<=nr<ROWS and 0<=nc<COLS and maze[nr][nc]==0 and (nr,nc) not in visited:
                visited.add((nr,nc)); dq.append((nr,nc))
    return visited

def get_valid_path_cells(maze,start,end):
    return bfs_reachable(maze,start).intersection(bfs_reachable(maze,end))

def generate_items(maze,start,end):
    valid=get_valid_path_cells(maze,start,end)
    valid.discard(start); valid.discard(end)
    pts=list(valid); random.shuffle(pts)
    items=[]
    for p in pts:
        if len(items)>=NUM_ITEMS: break
        if any(abs(p[0]-i.grid_pos[0])<ITEM_SPACING or abs(p[1]-i.grid_pos[1])<ITEM_SPACING for i in items): continue
        items.append(Item(p, random.choice(list(ITEM_TYPES.keys()))))
    return items

def generate_maze(rows,cols):
    while True:
        m=[[1]*cols for _ in range(rows)]
        start=(random.randrange(rows), random.randrange(cols//8))
        end=(random.randrange(rows), random.randrange(cols-cols//8,cols))
        if abs(start[1]-end[1])<MIN_DISTANCE: continue
        stack=[start]; m[start[0]][start[1]]=0
        dirs=[(-1,0),(1,0),(0,-1),(0,1)]
        while stack:
            r,c=stack[-1]; nbrs=[]
            for dr,dc in dirs:
                nr, nc = r+dr*2, c+dc*2
                if 0<=nr<rows and 0<=nc<cols and m[nr][nc]==1:
                    nbrs.append(((nr,nc),(r+dr,c+dc)))
            if nbrs:
                (nr,nc),(wr,wc)=random.choice(nbrs)
                m[wr][wc]=m[nr][nc]=0; stack.append((nr,nc))
            else: stack.pop()
        # random extra openings
        for _ in range(rows*cols//20):
            r,c=random.randrange(rows), random.randrange(cols)
            if m[r][c]==1 and sum(1 for dr,dc in dirs if 0<=r+dr<rows and 0<=c+dc<cols and m[r+dr][c+dc]==0)>0:
                m[r][c]=0
        if astar(m,start,end): return m,start,end

# --- Drawing Helpers ---
def draw_text(surf,text,pos,size=30,color=BLACK):
    f=pygame.font.SysFont(None,size)
    surf.blit(f.render(text,True,color),pos)

def update_camera(agent_x,dt):
    global camera_x
    tgt=max(0,min(agent_x-CAMERA_OFFSET, MAX_CAMERA_X))
    camera_x += (tgt-camera_x)*CAMERA_LERP*dt*60

# --- Game Initialization ---
maze, start, end = generate_maze(ROWS, COLS)
original_maze = [row[:] for row in maze]
path = astar(maze, start, end)
items = generate_items(maze, start, end)
agent = SmoothAgent(start)

game_over = False
camera_x = 0
last_block_time = datetime.now()
block_cooldown = 2
path_index = 0

# --- Main Game Loop ---
running = True
while running:
    dt = clock.tick(60) / 1000.0
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running=False
        elif event.type == pygame.MOUSEBUTTONDOWN and not game_over:
            mouse_x, mouse_y = pygame.mouse.get_pos()
            world_x = mouse_x + camera_x
            col = int(world_x//CELL_SIZE)
            row = int(mouse_y//CELL_SIZE)
            current_pos = (int(agent.current_pos.y//CELL_SIZE), int(agent.current_pos.x//CELL_SIZE))
            action=False
            if event.button==1 and 0<=row<ROWS and 0<=col<COLS and maze[row][col]==0 and (row,col) not in (start,end,current_pos):
                maze[row][col]=1; action=True
            elif event.button==3 and 0<=row<ROWS and 0<=col<COLS and maze[row][col]==1:
                maze[row][col]=0; action=True
            if action:
                place_block_sound.play()
                last_block_time=datetime.now()
                new_path=astar(maze,current_pos,end)
                if new_path:
                    path, path_index = new_path, 0
                    agent.target_pos = path[0]
                else:
                    game_over=True
                    pygame.mixer.music.stop()
                    game_over_sound.play()
        elif event.type==pygame.KEYDOWN:
            if event.key==pygame.K_r:
                maze=[r[:] for r in original_maze]
                path=astar(maze,start,end); items=generate_items(maze,start,end)
                agent=SmoothAgent(start); path_index=0; game_over=False; camera_x=0
                # (music/sound reset)
                pygame.mixer.music.play(-1)      # restart bg music
                game_over_sound.stop()           # stop any lingering game-over sound
            elif event.key==pygame.K_t:
                maze,start,end=generate_maze(ROWS,COLS)
                original_maze=[r[:] for r in maze]
                path=astar(maze,start,end); items=generate_items(maze,start,end)
                agent=SmoothAgent(start); path_index=0; game_over=False; camera_x=0
                # (music/sound reset)
                pygame.mixer.music.play(-1)      # restart bg music
                game_over_sound.stop()           # stop any lingering game-over sound

    # Update agent
    if not game_over and path and path_index < len(path):
        if agent.current_pos.distance_to(pygame.Vector2(path[path_index][1]*CELL_SIZE+CELL_SIZE/2,
                                              path[path_index][0]*CELL_SIZE+CELL_SIZE/2)) < max(1, agent.speed*dt*0.5):
            agent.current_pos = pygame.Vector2(path[path_index][1]*CELL_SIZE+CELL_SIZE/2,
                                              path[path_index][0]*CELL_SIZE+CELL_SIZE/2)
            path_index +=1
            if path_index < len(path): agent.target_pos=path[path_index]
        agent.update(path, dt)

    # Process effects
    agent.speed=agent.base_speed
    now=datetime.now()
    speed_mod=1.0; reveal=False
    for eff in list(agent.active_effects):
        info=ITEM_TYPES[eff['item_type_key']]
        if (now-eff['start_time']).total_seconds()>eff['duration']:
            agent.active_effects.remove(eff)
        else:
            if isinstance(info['effect'], (int,float)):
                speed_mod*=info['effect']
            elif info['effect'] and eff['item_type_key']=='reveal_path':
                reveal=True
    agent.speed=agent.base_speed*speed_mod
    show_path=reveal

    # Collision
    collided = Item.check_item_collision(agent.current_pos, items)
    if collided:
        pickup_sound.play()
        info=ITEM_TYPES[collided.type]
        exists=False
        for eff in agent.active_effects:
            if eff['item_type_key']==collided.type:
                eff['start_time']=now; exists=True; break
        if not exists:
            agent.active_effects.append({'item_type_key':collided.type,'start_time':now,'duration':info['duration']})

    # Camera
    update_camera(agent.current_pos.x, dt)

    # Draw
    screen.fill(LIGHT_BROWN)
    start_c=int(camera_x//CELL_SIZE); end_c=start_c+(WIDTH//CELL_SIZE)+2
    for r in range(ROWS):
        for c in range(start_c, min(COLS,end_c)):
            if maze[r][c]==1:
                screen.blit(wall_img, (c*CELL_SIZE-camera_x, r*CELL_SIZE))
    screen.blit(altar_img, (start[1]*CELL_SIZE-camera_x, start[0]*CELL_SIZE))
    screen.blit(blue_portal_img, (end[1]*CELL_SIZE-camera_x, end[0]*CELL_SIZE))
    for item in items:
        if item.active:
            img=ITEM_IMAGES[ITEM_TYPES[item.type]['image_key']]
            x=item.screen_pos[0]-camera_x-img.get_width()//2
            y=item.screen_pos[1]-img.get_height()//2
            screen.blit(img, (x,y))
    if show_path and not game_over:
        for i in range(path_index, len(path)):
            pr,pc=path[i]
            pygame.draw.circle(screen, ORANGE,(pc*CELL_SIZE-camera_x+CELL_SIZE//2,pr*CELL_SIZE+CELL_SIZE//2),3)
    ax=agent.current_pos.x-camera_x-thief_img.get_width()//2
    ay=agent.current_pos.y-thief_img.get_height()//2
    screen.blit(thief_img, (ax,ay))
    if (datetime.now()-last_block_time).total_seconds()<block_cooldown:
        rem=block_cooldown-(datetime.now()-last_block_time).total_seconds()
        draw_text(screen, f"Block Cooldown: {rem:.1f}s", (10,HEIGHT-40),24,RED)
    else:
        draw_text(screen, "Ready: LMB Add, RMB Remove", (10,HEIGHT-40),24,GREEN)
    draw_text(screen, f"Speed: {agent.speed/agent.base_speed:.2f}x", (WIDTH-150,10),24,BLACK)
    if game_over:
        s=pygame.Surface((WIDTH,HEIGHT),pygame.SRCALPHA)
        s.fill((50,50,50,180))
        screen.blit(s,(0,0))
        draw_text(screen,"Path Blocked or Goal Reached!",(WIDTH//2-200,HEIGHT//2-40),40,RED)
        draw_text(screen,"Press R to Restart or T for New Maze",(WIDTH//2-250,HEIGHT//2+10),30,WHITE)
    pygame.display.flip()

pygame.quit()
sys.exit()
